# 0. Import thư viện

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

# 1. Data Preprocessing
## 1.0. Data Loading

In [3]:
df = pd.read_csv('data/kill_match_stats.csv')

In [ ]:
df.head()

In [ ]:
df.drop(columns = ['killer_name', 'victim_name', 'killer_placement', 'victim_placement', 'match_id'], inplace = True)

In [ ]:
df = df[df['map'] == 'ERANGEL'].reset_index(drop = True)

In [ ]:
df.drop(columns = ['map'], inplace = True)

In [ ]:
df.rename(columns = {
    'killer_position_x': 'kx', 
    'killer_position_y': 'ky',
    'victim_position_x': 'vx',
    'victim_position_y': 'vy'
},inplace = True)

In [9]:
df.head()

,killed_by,kx,ky,time,vx,vy
0,Down and Out,496989.8,312569.7,1035,497385.4,331528.2
1,M16A4,496989.8,312569.7,1035,497819.4,331981.3
2,AKM,460416.7,414748.8,1422,459817.9,414426.3
3,AKM,488034.1,347220.3,1210,487444.2,347651.0
4,SKS,501062.9,425078.6,1818,493043.4,434458.1


<span style="color:red">Câu lệnh bên dưới chỉ thực hiện một lần duy nhất trên máy một thành viên duy nhất. Sau khi thực hiện, tiến hành lưu trữ df thành file kill_match_stats_v1.csv, các thành viên khác chỉ cần load file này vào là được, không cần chạy lại câu lệnh `to_csv` để tránh mất thời gian<span>

In [10]:
df.to_csv('data/kill_match_stats_v1.csv', index = False)

Test Markdown